In [1]:
import copernicusmarine
import xarray as xr
import fsspec
import pandas as pd

/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bio = xr.load_dataset(
    "/Users/ash/Documents/Workspaces/Data/cmems/2024-07 - 2x2 - LMTL no transport/cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_1723016267080.nc"
)
physic = xr.load_dataset(
    "/Users/ash/Documents/Workspaces/Data/cmems/2024-07 - 2x2 - LMTL no transport/cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i_1723015794520.nc"
)

In [3]:
data = xr.merge([bio, physic])
data

<xarray.Dataset> Size: 88MB
Dimensions:              (latitude: 24, longitude: 24, time: 1827, depth: 3)
Coordinates:
  * latitude             (latitude) float32 96B 0.08333 0.1667 ... 1.917 2.0
  * longitude            (longitude) float32 96B -180.0 -179.9 ... -178.2 -178.1
  * time                 (time) datetime64[ns] 15kB 2005-01-01 ... 2010-01-01
  * depth                (depth) float32 12B 1.0 2.0 3.0
Data variables: (12/13)
    mnkc_epi             (time, latitude, longitude) float32 4MB 0.1253 ... 0...
    mnkc_hmlmeso         (time, latitude, longitude) float32 4MB 0.2949 ... 0...
    mnkc_lmeso           (time, latitude, longitude) float32 4MB 3.764 ... 1.813
    mnkc_mlmeso          (time, latitude, longitude) float32 4MB 0.3108 ... 0...
    mnkc_mumeso          (time, latitude, longitude) float32 4MB 0.3088 ... 0...
    mnkc_umeso           (time, latitude, longitude) float32 4MB 0.1878 ... 0...
    ...                   ...
    zeu                  (time, latitude, longitude) float32 4MB 60.23 ... 82.53
    zooc                 (time, latitude, longitude) float32 4MB 0.13 ... 0.0...
    T                    (time, depth, latitude, longitude) float32 13MB 29.5...
    U                    (time, depth, latitude, longitude) float32 13MB -0.2...
    V                    (time, depth, latitude, longitude) float32 13MB 0.07...
    pelagic_layer_depth  (time, depth, latitude, longitude) float32 13MB 90.3...
Attributes:
    Conventions:       CF-1.11
    title:             Global ocean low and mid trophic levels biomass conten...
    institution:       CLS
    source:            SEAPODYM-LMTL 3.0.0
    history:           Created on 2022-09-28
    references:        http://www.cls.fr; http://www.seapodym.eu
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_202211
    subset:date:       2024-08-07T07:37:47.080Z

In [4]:
data = data[["T", "U", "V", "npp", "zeu", "pelagic_layer_depth"]]
data

<xarray.Dataset> Size: 59MB
Dimensions:              (time: 1827, depth: 3, latitude: 24, longitude: 24)
Coordinates:
  * latitude             (latitude) float32 96B 0.08333 0.1667 ... 1.917 2.0
  * longitude            (longitude) float32 96B -180.0 -179.9 ... -178.2 -178.1
  * time                 (time) datetime64[ns] 15kB 2005-01-01 ... 2010-01-01
  * depth                (depth) float32 12B 1.0 2.0 3.0
Data variables:
    T                    (time, depth, latitude, longitude) float32 13MB 29.5...
    U                    (time, depth, latitude, longitude) float32 13MB -0.2...
    V                    (time, depth, latitude, longitude) float32 13MB 0.07...
    npp                  (time, latitude, longitude) float32 4MB 400.9 ... 188.9
    zeu                  (time, latitude, longitude) float32 4MB 60.23 ... 82.53
    pelagic_layer_depth  (time, depth, latitude, longitude) float32 13MB 90.3...
Attributes:
    Conventions:       CF-1.11
    title:             Global ocean low and mid trophic levels biomass conten...
    institution:       CLS
    source:            SEAPODYM-LMTL 3.0.0
    history:           Created on 2022-09-28
    references:        http://www.cls.fr; http://www.seapodym.eu
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_202211
    subset:date:       2024-08-07T07:37:47.080Z

In [5]:
data["U"] = data["U"].where(data["U"].isnull(), 0)
data["V"] = data["V"].where(data["U"].isnull(), 0)
data

<xarray.Dataset> Size: 59MB
Dimensions:              (time: 1827, depth: 3, latitude: 24, longitude: 24)
Coordinates:
  * latitude             (latitude) float32 96B 0.08333 0.1667 ... 1.917 2.0
  * longitude            (longitude) float32 96B -180.0 -179.9 ... -178.2 -178.1
  * time                 (time) datetime64[ns] 15kB 2005-01-01 ... 2010-01-01
  * depth                (depth) float32 12B 1.0 2.0 3.0
Data variables:
    T                    (time, depth, latitude, longitude) float32 13MB 29.5...
    U                    (time, depth, latitude, longitude) float32 13MB 0.0 ...
    V                    (time, depth, latitude, longitude) float32 13MB 0.0 ...
    npp                  (time, latitude, longitude) float32 4MB 400.9 ... 188.9
    zeu                  (time, latitude, longitude) float32 4MB 60.23 ... 82.53
    pelagic_layer_depth  (time, depth, latitude, longitude) float32 13MB 90.3...
Attributes:
    Conventions:       CF-1.11
    title:             Global ocean low and mid trophic levels biomass conten...
    institution:       CLS
    source:            SEAPODYM-LMTL 3.0.0
    history:           Created on 2022-09-28
    references:        http://www.cls.fr; http://www.seapodym.eu
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_202211
    subset:date:       2024-08-07T07:37:47.080Z

In [7]:
data.to_netcdf("/Users/ash/Documents/Workspaces/Data/phd/SEAPODYM_LMTL/data.nc")